In [ ]:
import json

from dash import Dash, Input, Output, State, dcc, html
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import openai
import tiktoken
import plotly, jupyter_dash, dash
openai.api_key = 'YOU API KEY'

openai_model = 'YOUR MODEL NAME'
for pkg in [openai, pd, dash, plotly, dbc, plotly, jupyter_dash]:
    print(f'{pkg.__name__:-<30}v{pkg.__version__}')

openai------------------------v0.27.5
pandas------------------------v2.0.2
dash--------------------------v2.9.1
plotly------------------------v5.13.1
dash_bootstrap_components-----v1.3.1
plotly------------------------v5.13.1
jupyter_dash------------------v0.4.2


In [ ]:
def get_completion(prompt, temperature=0, model=openai_model):
    response = openai.Completion.create(
        prompt = prompt + prompt_end,
        max_tokens=500,
        stop=[completion_end, prompt_end],
        model=model,
        temperature=0)
    return response
    comp =  response.choices[0]['text']
    print('#' * 80)
    import datetime
    print(datetime.datetime.now())
    print(comp)
    print('#' * 80)
    return comp
    result_df = pd.DataFrame(
        [entity.strip().split(': ') for entity in  comp.split('@@')],
        columns=['entity', 'wikipedia_url'])
    return result_df


In [ ]:
prompt_end = '\n\n%%%%%\n'
completion_end = '\n\n^^^^^\n'


def get_completion(prompt, temperature=0, model=openai_model):
    response = openai.Completion.create(
        prompt = prompt + prompt_end,
        max_tokens=500,
        stop=[completion_end, prompt_end],
        model=model,
        temperature=0)
    comp =  response.choices[0]['text']
    result_df = pd.DataFrame(
        [entity.strip().split(': ') for entity in  comp.split('@@')],
        columns=['entity', 'wikipedia_url'])
    return result_df


def openai_resp_to_json(resp_df):
    script_open = '<script type="application/ld+json">'
    schema_list = [{
        "@context": "https://schema.org",
        "@type": "Thing",
        "name": entity,
        "url": url}
        for entity, url in resp_df.values]
    return script_open +  json.dumps(schema_list) + '</script>'


def split_prompt(prompt, max_tokens=1500, model='davinci'):
    encoder = tiktoken.encoding_for_model(model)
    encoded_txt = encoder.encode(prompt)
    split_txt = [encoded_txt[x:x + max_tokens]
                 for x in range(0, len(encoded_txt), max_tokens)]
    decoded = [encoder.decode(t) for t in split_txt]
    return decoded


In [ ]:
app = Dash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = html.Div([
    dbc.Row([
        dbc.Col(lg=1),
        dbc.Col([
            html.Br(), html.Br(),
            dbc.Label('Paste an article:'),
            dbc.Textarea(id='user_text', rows=15), html.Br(),
            dbc.Button('Extract entities', id='submit_button'), html.Br(),html.Br(),html.Br(),
            dcc.Loading(html.Div(id='output'))
        ] + [html.Br() for i in range(15)], lg=10)
    ])
])


@app.callback(
    Output('output', 'children'),
    Input('submit_button', 'n_clicks'),
    State('user_text', 'value'))
def extract_entites(n_clicks, user_text):
    if not n_clicks:
        raise PreventUpdate
    user_text = split_prompt(user_text)
    openai_responses = []
    for prompt in user_text:
        openai_resp = get_completion(prompt)
        openai_responses.append(openai_resp)
    df = pd.concat(openai_responses, ignore_index=True).drop_duplicates()
    table = dbc.Table.from_dataframe(
        df.rename(columns={'entity': 'Entity',
                           'wikipedia_url': 'Wikipedia URL'}))
    return html.Div(dbc.Tabs([
        dbc.Tab(table, label='Table'),
        dbc.Tab([
            html.Br(),
            html.Code(openai_resp_to_json(openai_resp))],
            label='JSON-LD')
    ]))


app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
